In [1]:
from skimage import io, measure, util, transform, filters
from pathlib import Path
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
%matplotlib qt 

In [37]:
path = Path("..") / "data" / "results" 
image_paths = path.glob("*.png")
for image_path in image_paths:
    print(image_path)
    img = io.imread(image_path)
    print(Counter(img.ravel()))
    print(img.shape)
    io.imshow(img)
    
#     img_label = measure.label(img)
#     regions = measure.regionprops(img_label)
#     print("liczba regionów:", len(regions))
#     mean = np.mean([reg.area for reg in regions])/img.shape[1]
#     std = np.std([reg.area for reg in regions])/img.shape[1]
#     max_val = np.max([reg.area for reg in regions])/img.shape[1]
#     min_val = np.min([reg.area for reg in regions])/img.shape[1]
#     print(mean, std, max_val, min_val)
    
#     regions = [reg for reg in regions if reg.area/img.shape[1] > std]
#     print("POZOSTAŁA liczba regionów:", len(regions))
# #     for reg in regions:
# #         print(reg.area/img.shape[1])
    
    
#     print("###################################")

..\data\results\1-wyrazy.png
Counter({0: 3677070, 1: 255090})
(2560, 1536)
..\data\results\10-wyrazy.png
Counter({0: 3713978, 1: 218182})
(2560, 1536)


AttributeError: 'NoneType' object has no attribute 'get_topmost_subplotspec'

# OBRACANIE

In [154]:
def thresholding(img):
    thresh = filters.threshold_otsu(img, 3)
    img = img > thresh
    return img

def resize_after_rotate(img_rotated, destination_shape):
    diff = (np.array(img_rotated.shape) - np.array(destination_shape))/2
    
    if diff[0] == 0.0 and diff[1] == 0.0:
        new = img_rotated
    elif diff[0] == 0.0:
        new = img_rotated[:, int(diff[1]):]
    elif diff[1] == 0.0:
        new = img_rotated[int(diff[0]):, :]
    else:
        new = img_rotated[int(diff[0]):-int(diff[0]), int(diff[1]):-int(diff[1])]

    diff = (np.array(new.shape) - np.array(destination_shape))
    if diff[0] > 0:
        new = new[:-diff[0], :]
    
    if diff[1] > 0:
        new = new[:, :-diff[1]]

    new = new.astype(np.uint8)
    return new

#     diff = (np.array(img_rotated.shape) - np.array(destination_shape))/2
#     new = img_rotated[int(diff[0]):-int(diff[0]), int(diff[1]):-int(diff[1])]
    
#     diff = (np.array(new.shape) - np.array(destination_shape))
#     if diff[0] > 0:
#         new = new[:-diff[0]]
    
#     if diff[1] > 0:
#         new = new[:, :-diff[1]]
#     new = thresholding(new)
#     return new


    

In [156]:
# %matplotlib inline
pp = Path('../data/partial_results/1/2_kontury_wyrazow_na_fragmencie/3.png')
img = io.imread(pp)
img_rot = transform.rotate(img, 0,  resize=True, preserve_range=True)
img_rot_inv = transform.rotate(img_rot, -0,  resize=True, preserve_range=True)

In [157]:
print(img.shape)
io.imshow(img)

(1290, 910)


In [158]:
print(img_rot.shape)
io.imshow(img_rot)

(1290, 910)


In [159]:
print(img_rot_inv.shape)
io.imshow(img_rot_inv)

(1290, 910)


In [78]:
diff = (np.array(img_rot_inv.shape) - np.array(img.shape))/2
new = img_rot_inv[int(diff[0]):-int(diff[0]), int(diff[1]):-int(diff[1])]
print(new.shape)
io.imshow(new)

(1291, 910)


d:\dokumenty\projekty\piro\venv\lib\site-packages\skimage\io\_plugins\matplotlib_plugin.py:150: UserWarning: Float image out of standard range; displaying image with stretched contrast.
  lo, hi, cmap = _get_display_range(image)


In [160]:
w = resize_after_rotate(img_rot_inv, img.shape)
print(w.shape)
io.imshow(w)

(1290, 910)


In [49]:
print(img.shape)
print(img_rot.shape)
print(img_rot_inv.shape)
print(new.shape)

(1290, 910)
(1526, 1573)
(2185, 2180)
(1291, 910)


# łączenie linii

In [67]:
def find_closest_region(all_regions, region):
    """
        Wyszukuje najbliższego regionu względem osi Y (wysokość obrazu).
        
    """
    region_y_min = np.min(region.coords[:, 0])
    region_y_max = np.max(region.coords[:, 0])
    
    temp_distance = np.inf
    closest_region = None
    for reg in all_regions:
        current_reg_y_min = np.min(reg.coords[:, 0])
        current_reg_y_max = np.max(reg.coords[:, 0])
        
        is_reg_higher = True
        if current_reg_y_min > region_y_min:
            is_reg_higher = False
        
        if is_reg_higher:
            diff = region_y_min - current_reg_y_max
        else:
            diff = current_reg_y_min - region_y_max

        if diff < temp_distance:
            temp_distance = diff
            closest_region = reg
            
    return closest_region

In [136]:
from skimage import color

path = Path("..") / "data" / "partial_results" / "3" / "1_wykryte_wiersze_tekstu" 
image_paths = path.glob("*_1.png")

save_path = Path('../data/partial_results/TEST')
save_path.mkdir(parents=True, exist_ok=True)
for image_path in image_paths:
#     image_path = path / "25_1.png"
    print(image_path)
    img = io.imread(image_path)

    
    img_label = measure.label(img)
    regions = measure.regionprops(img_label)
    
    std = np.std([reg.area for reg in regions])/img.shape[1]


    big_regions = [reg for reg in regions if reg.area/img.shape[1] > std*0.75]
    small_regions = [reg for reg in regions if reg.area/img.shape[1] <= std*0.75]

    for reg in small_regions:
        closest_reg = find_closest_region(all_regions=big_regions, region=reg)
        
        closest_reg_height = np.max(closest_reg.coords[:, 0])
        reg_height = np.max(reg.coords[:, 0])
        
        img[min(reg_height, closest_reg_height):max(reg_height, closest_reg_height)+1, :] = np.iinfo(img.dtype).max
        
    img_inv = util.invert(img)
    img_label = measure.label(img_inv)
    regions = measure.regionprops(img_label)[1:-1] # po odwórceniu nie chcemy górnej i dolnej krawędzi
    

    std = np.std([reg.area for reg in regions])/img.shape[1]
    print(std)
    for reg in regions:
        print(reg.area/img.shape[1])

    img = color.gray2rgb(img)
    small_regions = [reg for reg in regions if reg.area/img.shape[1] <= std*0.5]

    for reg in small_regions:
        p1 = reg.coords[0]
        p2 = reg.coords[-1]
        img[p1[0]:p2[0]+1, p1[1]:p2[1]+1] = np.iinfo(img.dtype).max
    
#     io.imshow(img)
#     break
    

    io.imsave(arr=img, fname=save_path / (image_path.stem+'.png'))

        




..\data\partial_results\3\1_wykryte_wiersze_tekstu\10_1.png
4.842989309769893
41.0
38.0
43.0
39.0
34.0
33.0
34.0
32.0
28.0
42.0
43.0
..\data\partial_results\3\1_wykryte_wiersze_tekstu\11_1.png
26.06549443229497
5.0
76.0
98.0
52.0
47.0
47.0
47.0
95.0
47.0
43.0
..\data\partial_results\3\1_wykryte_wiersze_tekstu\12_1.png
4.987587899761892
44.0
40.0
45.0
38.0
37.0
34.0
40.0
36.0
32.0
42.0
50.0
..\data\partial_results\3\1_wykryte_wiersze_tekstu\13_1.png
11.790250209389113
1.0
26.0
39.0
34.0
29.0
34.0
32.0
8.0
8.0
32.0
31.0
38.0
32.0
32.0
27.0
36.0
12.0
47.0
40.0
36.0
..\data\partial_results\3\1_wykryte_wiersze_tekstu\14_1.png
6.269906569882978
48.0
29.0
49.0
46.0
34.0
41.0
33.0
45.0
37.0
33.0
41.0
43.0
47.0
36.0
45.0
49.0
49.0
38.0
..\data\partial_results\3\1_wykryte_wiersze_tekstu\15_1.png
11.753453420028787
56.0
34.0
39.0
37.0
32.0
36.0
32.0
26.0
29.0
32.0
31.0
52.0
38.0
44.0
35.0
36.0
36.0
43.0
20.0
12.0
22.0
6.0
12.0
..\data\partial_results\3\1_wykryte_wiersze_tekstu\16_1.png
10.2414598

255